In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
transaction = pd.read_csv('./data/transaction_new.csv')
customer = pd.read_csv('./data/customer.csv')


#transaction = pd.read_csv('transaction_new.csv')
#customer = pd.read_csv('customer.csv')

In [3]:
customer = customer.assign(first_join_date=pd.to_datetime(customer['first_join_date']))
transaction = transaction.assign(created_at=pd.to_datetime(transaction['created_at']))
transaction = transaction.assign(created_at = transaction['created_at'].dt.strftime('%Y-%m-%d'))
transaction = transaction.assign(created_at = pd.to_datetime(transaction['created_at']))
transaction = transaction.assign(total_price=transaction['total_amount'] - transaction['shipment_fee'])

In [4]:
paid_transaction = transaction[transaction.payment_status=='Success']

In [5]:
customer_date = customer[['customer_id', 'first_join_date']].sort_values('first_join_date').reset_index(drop=True)
customer_date

,customer_id,first_join_date
0,74089,2016-06-30
1,21734,2016-06-30
2,38119,2016-06-30
3,73773,2016-06-30
4,60006,2016-06-30
...,...,...
99995,73572,2022-07-31
99996,66776,2022-07-31
99997,96644,2022-07-31
99998,27995,2022-07-31


* ARPU
    * 매출(주 단위) / 주 단위로 누적(유입) 유저
* ARPPU
    * 매출(주 단위) / 주 단위로 누적 구매 유저

In [6]:
transaction_by_session = paid_transaction.groupby('session_id')[['created_at', 'customer_id', 'total_price']].max().sort_values('created_at').reset_index(drop=True)
transaction_by_session

,created_at,customer_id,total_price
0,2016-06-30,74089,630795
1,2016-07-01,73773,246670
2,2016-07-01,16228,318792
3,2016-07-01,6183,568826
4,2016-07-02,30771,194897
...,...,...,...
815959,2022-07-31,65263,186150
815960,2022-07-31,76961,158899
815961,2022-07-31,95196,390873
815962,2022-07-31,7748,568644


In [35]:
data_start_date = pd.to_datetime('2016-06-30')
previous_start_date = pd.to_datetime('2016-07-10')
previous_end_date = previous_start_date + timedelta(days=6)

current_start_date = previous_start_date + timedelta(days=7)
current_end_date = previous_start_date + timedelta(days=13)

print(previous_start_date, previous_end_date)
print(current_start_date, current_end_date)

2016-07-10 00:00:00 2016-07-16 00:00:00
2016-07-17 00:00:00 2016-07-23 00:00:00


In [36]:
prev_customer = customer_date[(customer_date.first_join_date>=data_start_date)&
                              (customer_date.first_join_date<=previous_end_date)]

curr_customer = customer_date[(customer_date.first_join_date>=data_start_date)&
                              (customer_date.first_join_date<=current_end_date)]

In [37]:
prev_transaction = transaction_by_session[(transaction_by_session.created_at>=previous_start_date)&
                                          (transaction_by_session.created_at<=previous_end_date)]

curr_transaction = transaction_by_session[(transaction_by_session.created_at>=current_start_date)&
                                          (transaction_by_session.created_at<=current_end_date)]

prev_transaction_cum = transaction_by_session[(transaction_by_session.created_at>=data_start_date)&
                                              (transaction_by_session.created_at<=previous_end_date)]

curr_transaction_cum = transaction_by_session[(transaction_by_session.created_at>=data_start_date)&
                                              (transaction_by_session.created_at<=current_end_date)]

In [38]:
prev_transaction.groupby('created_at')['total_price'].sum().cumsum()

created_at
2016-07-10     7812393
2016-07-11    11884816
2016-07-12    14201841
2016-07-13    16835266
2016-07-14    19140272
2016-07-15    20128945
2016-07-16    21952100
Name: total_price, dtype: int64

In [39]:
prev_customer.groupby('first_join_date')['customer_id'].nunique().cumsum()

first_join_date
2016-06-30      6
2016-07-01     16
2016-07-02     51
2016-07-03     87
2016-07-04    102
2016-07-05    106
2016-07-06    116
2016-07-07    125
2016-07-08    132
2016-07-09    215
2016-07-10    281
2016-07-11    290
2016-07-12    298
2016-07-13    306
2016-07-14    313
2016-07-15    320
2016-07-16    385
Name: customer_id, dtype: int64

In [40]:
prev_transaction.total_price.sum()

21952100

In [41]:
prev_customer.customer_id.nunique()

385

In [13]:
# ARPU

print('prev', prev_transaction.total_price.sum() / prev_customer.customer_id.nunique())
print('curr', curr_transaction.total_price.sum() / curr_customer.customer_id.nunique())

prev 59571.40838852097
curr 59987.321202531646


In [14]:
# ARPPU
print('prev', prev_transaction.total_price.sum() / prev_transaction_cum.customer_id.nunique())
print('curr', curr_transaction.total_price.sum() / curr_transaction_cum.customer_id.nunique())

prev 186109.29655172414
curr 185843.07352941178


* ARPU
    * 매출(월 단위) / 월 단위로 누적(유입) 유저
* ARPPU
    * 매출(주 단위) / 월 단위로 누적 구매 유저

In [15]:
customer_date = customer_date.assign(first_join_month = customer_date['first_join_date'].dt.strftime('%Y-%m'))
transaction_by_session = transaction_by_session.assign(purchase_month = transaction_by_session['created_at'].dt.strftime('%Y-%m'))

display(customer_date)
display(transaction_by_session)

,customer_id,first_join_date,first_join_month
0,74089,2016-06-30,2016-06
1,21734,2016-06-30,2016-06
2,38119,2016-06-30,2016-06
3,73773,2016-06-30,2016-06
4,60006,2016-06-30,2016-06
...,...,...,...
99995,73572,2022-07-31,2022-07
99996,66776,2022-07-31,2022-07
99997,96644,2022-07-31,2022-07
99998,27995,2022-07-31,2022-07


,created_at,customer_id,total_price,purchase_month
0,2016-06-30,74089,630795,2016-06
1,2016-07-01,73773,246670,2016-07
2,2016-07-01,16228,318792,2016-07
3,2016-07-01,6183,568826,2016-07
4,2016-07-02,30771,194897,2016-07
...,...,...,...,...
815959,2022-07-31,65263,186150,2022-07
815960,2022-07-31,76961,158899,2022-07
815961,2022-07-31,95196,390873,2022-07
815962,2022-07-31,7748,568644,2022-07


In [16]:
FOR_USER = customer_date[customer_date['first_join_month']<='2017-01']
FOR_PU = transaction_by_session[transaction_by_session['purchase_month']<='2017-01']
FOR_REVENUE = transaction_by_session[transaction_by_session['purchase_month'].str.contains('2017')]

In [17]:
FOR_USER

,customer_id,first_join_date,first_join_month
0,74089,2016-06-30,2016-06
1,21734,2016-06-30,2016-06
2,38119,2016-06-30,2016-06
3,73773,2016-06-30,2016-06
4,60006,2016-06-30,2016-06
...,...,...,...
5540,87033,2017-01-31,2017-01
5541,37491,2017-01-31,2017-01
5542,3127,2017-01-31,2017-01
5543,87539,2017-01-31,2017-01


In [18]:
USER = FOR_USER.customer_id.nunique()
USER

5545

In [19]:
FOR_PU

,created_at,customer_id,total_price,purchase_month
0,2016-06-30,74089,630795,2016-06
1,2016-07-01,73773,246670,2016-07
2,2016-07-01,16228,318792,2016-07
3,2016-07-01,6183,568826,2016-07
4,2016-07-02,30771,194897,2016-07
...,...,...,...,...
6085,2017-01-31,65610,247810,2017-01
6086,2017-01-31,23886,561484,2017-01
6087,2017-01-31,48647,126812,2017-01
6088,2017-01-31,80637,1465472,2017-01


In [20]:
PU = FOR_PU.customer_id.nunique()
PU

2905

In [21]:
REVENUE = FOR_REVENUE.groupby('purchase_month')['total_price'].sum().to_frame().reset_index()
REVENUE

,purchase_month,total_price
0,2017-01,746139699
1,2017-02,789214663
2,2017-03,1040794092
3,2017-04,1060049088
4,2017-05,1272721448
5,2017-06,1193512723
6,2017-07,1735658236
7,2017-08,1671634960
8,2017-09,1836364475
9,2017-10,2044204910


In [22]:
# ARPPU

REVENUE[REVENUE['purchase_month']=='2017-01']['total_price'] / PU

0    256846.71222
Name: total_price, dtype: float64

In [23]:
784823105 / 2899

270722.0093135564

In [24]:
# ARPU

REVENUE[REVENUE['purchase_month']=='2017-01']['total_price'] / USER

0    134560.811362
Name: total_price, dtype: float64

In [25]:
784823105 / 5545

141537.07935076646